In [ ]:
import os
import pandas as pd

directory_path = '/content/drive/My Drive/final'
excel_files = [f for f in os.listdir(directory_path) if f.endswith('.xlsx') or f.endswith('.xls')]

dataframes = {}

for file in excel_files:
    file_path = os.path.join(directory_path, file)
    df = pd.read_excel(file_path)
    dataframes[file] = df

In [ ]:
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout

# Path to your data
base_dir = '/content/drive/My Drive/final'

# Get a list of all images and their class names
data = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.jpg'):  # adjust this as necessary
            data.append((os.path.join(root, file), os.path.basename(root)))

# Convert to a DataFrame
data = pd.DataFrame(data, columns=['filename', 'class'])

# Split the data
train_data, temp_data = train_test_split(data, test_size=0.6, stratify=data['class'])
val_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['class'])

# Preprocessing functions
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [150, 150])
    image /= 255.0  # normalize to [0,1] range
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

# Create a label index
label_names = train_data['class'].unique().tolist()
label_to_index = dict((name, index) for index, name in enumerate(label_names))
index_to_label = dict((index, name) for index, name in enumerate(label_names))

# Convert class names to class indices
train_data['class'] = train_data['class'].map(label_to_index)
val_data['class'] = val_data['class'].map(label_to_index)
test_data['class'] = test_data['class'].map(label_to_index)

# Create datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_data['filename'], train_data['class'])).map(load_and_preprocess_from_path_label)
val_ds = tf.data.Dataset.from_tensor_slices((val_data['filename'], val_data['class'])).map(load_and_preprocess_from_path_label)
test_ds = tf.data.Dataset.from_tensor_slices((test_data['filename'], test_data['class'])).map(load_and_preprocess_from_path_label)

# Batch and prefetch
train_ds = train_ds.batch(32).prefetch(1)
val_ds = val_ds.batch(32).prefetch(1)
test_ds = test_ds.batch(32).prefetch(1)

# Load the VGG16 model but exclude the top layers
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(150, 150, 3))

# Freeze the base_model
base_model.trainable = False

# Create a new model on top
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(label_names), activation='softmax')
])

# You might want to fine-tune some layers
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Use a smaller learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_ds, verbose=1)
print('\nTest accuracy:', test_acc)

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
60/60 [==============================] - 959s 16s/step - loss: 4.0825 - accuracy: 0.0818 - val_loss: 3.5641 - val_accuracy: 0.2383
Epoch 2/10
60/60 [==============================] - 885s 15s/step - loss: 3.2472 - accuracy: 0.2549 - val_loss: 2.9115 - val_accuracy: 0.3557
Epoch 3/10
60/60 [==============================] - 877s 15s/step - loss: 2.5270 - accuracy: 0.4111 - val_loss: 2.4656 - val_accuracy: 0.4270
Epoch 4/10
60/60 [==============================] - 874s 15s/step - loss: 1.9879 - accuracy: 0.5186 - val_loss: 2.1753 - val_accuracy: 0.4710
Epoch 5/10
60/60 [==============================] - 877s 15s/step - loss: 1.5878 - accuracy: 0.6303 - val_loss: 1.9210 - val_accuracy: 0.5213
Epoch 6/10
60/60 [==============================] - 873s 15s/step - loss: 1.2721 - accuracy: 0.7074 - val_loss: 1.7833 - val_accuracy: 0.5437
Epoch 7/10
60/60 [==============================] - 872s 15s/step - loss: 1.0299 - 

In [ ]:
img_path = '/content/drive/My Drive/Segmented Medicinal Leaf Images 2/carisa.webp'  # replace with your actual path

In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

# Load the image for displaying
img_display = image.load_img(img_path)

# Load the image for prediction and resize
img = image.load_img(img_path, target_size=(64, 64))  # Adjust target_size to match your model's input shape
img_array = image.img_to_array(img)
img_array_expanded = np.expand_dims(img_array, axis=0)
img_array_expanded /= 255.  # If your model's training input was normalized, do the same for prediction

# Display the image
plt.imshow(img_display)
plt.show()

# Make a prediction
prediction = model.predict(img_array_expanded)
predicted_class = np.argmax(prediction[0], -1)

# Get the class names from the generator
class_names = list(train_generator.class_indices.keys())
plant_name = class_names[predicted_class]

print("The predicted plant is:", plant_name)

In [ ]:
import matplotlib.pylab as plt

# Extract accuracy and loss from history object
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))

# Plot training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")

plt.show()

In [ ]:
# Save the model
model.save('my_model.h5')

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")